In [4]:
import os
import pandas as pd
import janitor

dfs = {}

ruta = os.path.join('..', 'data')

years = range(2019, 2024)

for year in years:
    file_path = os.path.join(ruta, f'{year}clean.csv')  # crea la ruta al archivo
    dfs[f'df{year}'] = pd.read_csv(file_path)  # lee el archivo CSV y lo almacena en el diccionario

df2019 = dfs['df2019']
df2020 = dfs['df2020']
df2021 = dfs['df2021']
df2022 = dfs['df2022']
df2023 = dfs['df2023']

for name, df in dfs.items():
    dfs[name] = df.clean_names()

/var/folders/n1/5l3l1c6x5xd7lfxmx1cmr21c0000gn/T/ipykernel_74477/2312118806.py:13: DtypeWarning: Columns (6,7,20,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[f'df{year}'] = pd.read_csv(file_path)  # lee el archivo CSV y lo almacena en el diccionario
/var/folders/n1/5l3l1c6x5xd7lfxmx1cmr21c0000gn/T/ipykernel_74477/2312118806.py:13: DtypeWarning: Columns (6,7,20,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[f'df{year}'] = pd.read_csv(file_path)  # lee el archivo CSV y lo almacena en el diccionario
/var/folders/n1/5l3l1c6x5xd7lfxmx1cmr21c0000gn/T/ipykernel_74477/2312118806.py:13: DtypeWarning: Columns (6,7,20,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[f'df{year}'] = pd.read_csv(file_path)  # lee el archivo CSV y lo almacena en el diccionario
/var/folders/n1/5l3l1c6x5xd7lfxmx1cmr21c0000gn/T/ipykernel_74477/2312118806.py:13: DtypeWarning: Columns (6,20,42,43) have m

In [11]:
# Crear una lista de dataframes
dfs_list = [dfs[name] for name in dfs.keys()]

# Concatenar todos los dataframes
contratos_df = pd.concat(dfs_list, ignore_index=True)


In [12]:
from currency_converter import CurrencyConverter


for column in contratos_df.columns:
    if contratos_df[column].dtype == 'object':
        contratos_df[column] = contratos_df[column].str.upper()


for column in contratos_df.columns:
    if contratos_df[column].dtype == 'string':
        contratos_df[column] = contratos_df[column].str.upper()

c = CurrencyConverter()
currencies=['USD', 'CAD', 'EUR', 'GBP', 'JPY']

# Create a dictionary to store conversion rates
conversion_rates = {}

for m in currencies:
    conversion_rate = c.convert(1, m, 'MXN')
    print(1, m, "=", conversion_rate, 'MXN')
    conversion_rates[m] = conversion_rate

# Apply conversion rates
contratos_df["importe"] = contratos_df["importe_del_contrato"]

for currency, conversion_rate in conversion_rates.items():
    contratos_df.loc[contratos_df['moneda_del_contrato'] == currency , 'importe'] = contratos_df.loc[contratos_df['moneda_del_contrato'] == currency , "importe_del_contrato"] * conversion_rate

print('Done')


1 USD = 17.115130976369297 MXN
1 CAD = 13.018183084854396 MXN
1 EUR = 18.6863 MXN
1 GBP = 21.75912340762477 MXN
1 JPY = 0.11946998273767662 MXN
Done


### Función de búsqueda

In [17]:
def filter_contratos(contratos_df):
    # Solicita al usuario un proveedor o contratista o RFC
    search_term = input("Por favor, ingresa el nombre del proveedor o contratista o el RFC (déjalo en blanco para mostrar todos): ").lower()

    # Solicita al usuario las siglas de la institución o "todas" para buscar en todas
    institution = input("Por favor, ingresa las siglas de la institución (déjalo en blanco para mostrar todas) o 'todas' para buscar en todas: ").upper()

    # Solicita al usuario las palabras clave en la descripción del contrato
    keywords = input("Por favor, ingresa las palabras clave en la descripción del contrato (separa múltiples palabras con comas, déjalo en blanco para buscar todas): ").upper()

    # Solicita al usuario las palabras a excluir en la descripción del contrato
    exclude_words = input("Por favor, ingresa las palabras a excluir en la descripción del contrato (separa múltiples palabras con comas, déjalo en blanco para no excluir ninguna): ").upper()

    # Inicialmente asumimos todos los contratos
    filtered_df = contratos_df.copy()  # Creamos una copia del DataFrame

    filtered_df[['proveedor_o_contratista', 'descripcion_del_contrato']] = filtered_df[['proveedor_o_contratista', 'descripcion_del_contrato']].fillna('')


# Convierte las columnas de texto a minúsculas
    filtered_df['proveedor_o_contratista'] = filtered_df['proveedor_o_contratista'].str.upper()
    filtered_df['siglas_de_la_institucion'] = filtered_df['siglas_de_la_institucion'].str.upper()
    filtered_df['descripcion_del_contrato'] = filtered_df['descripcion_del_contrato'].str.upper()

    # Filtra por proveedor o contratista o RFC si se proporciona
    if search_term:
        if len(search_term) == 12 and search_term.isalnum(): # RFC en México tiene 13 caracteres y sólo contiene letras y números
            filtered_df = filtered_df.loc[filtered_df['rfc'] == search_term]
        else:
            filtered_df = filtered_df.loc[filtered_df['proveedor_o_contratista'].str.contains(search_term, case=False)]

    # Filtra por institución si se proporciona
    if institution != 'todas' and institution:
        filtered_df = filtered_df.loc[filtered_df['siglas_de_la_institucion'] == institution]

    # Reemplaza los NaN en la descripción del contrato por cadenas vacías
    filtered_df['descripcion_del_contrato'] = filtered_df['descripcion_del_contrato'].fillna('')

    # Filtra por palabras clave en la descripción del contrato si se proporcionan
    if keywords:
        for keyword in keywords.split(','):
            keyword = keyword.strip()  # Remueve los espacios adicionales al principio y al final de la palabra
            filtered_df = filtered_df.loc[filtered_df['descripcion_del_contrato'].str.contains(keyword, case=False)]

    # Excluye contratos que contienen ciertas palabras en la descripción del contrato si se proporcionan
    if exclude_words:
        for word in exclude_words.split(','):
            word = word.strip()  # Remueve los espacios adicionales al principio y al final de la palabra
            filtered_df = filtered_df.loc[~filtered_df['descripcion_del_contrato'].str.contains(word, case=False)]

    # Ordena el DataFrame filtrado por importe en orden descendente
    filtered_df = filtered_df.sort_values(by='importe', ascending=False)

    # Muestra el DataFrame filtrado
    return filtered_df

# Usa la función:


# Búsqueda

In [27]:
# Usa la función:
buscar = filter_contratos(contratos_df)
buscar[['siglas_de_la_institucion', 'titulo_del_contrato', 'descripcion_del_contrato','rfc','proveedor_o_contratista', 'importe', 'direccion_del_anuncio' ]]

,siglas_de_la_institucion,titulo_del_contrato,descripcion_del_contrato,rfc,proveedor_o_contratista,importe,direccion_del_anuncio
58417,API-VERACRUZ,SERVICIO PARA DICTAMINAR LOS ESTADOS FINANCIE...,SERVICIO PARA DICTAMINAR LOS ESTADOS FINANCIER...,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","344,885.0",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...
311405,API-TUXPAN,AUDITORIA EXTERNA PARA DICTAMINAR LOS ESTADOS ...,AUDITORIA EXTERNA PARA DICTAMINAR LOS ESTADOS ...,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","311,986.0",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...
65308,API-TUXPAN,CONTRATO ABIERTO DE PRESTACIÓN DE SERVICIOS PA...,CONTRATO ABIERTO DE PRESTACIÓN DE SERVICIOS PA...,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","307,432.0",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...
185925,API-SALINA CRUZ,AUDITORIA EXTERNA DE ESTADOS FINANCIEROS 2020,AUDITORIA EXTERNA DE ESTADOS FINANCIEROS CON C...,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","305,502.0",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...
211734,API-TUXPAN,CONTRATO ABIERTO PARA LA REALIZACIÓN DE DICTAM...,CONTRATO ABIERTO DE PRESTACIÓN DE SERVICIOS PA...,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","305,102.0",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...
51826,API-SALINA CRUZ,SERVICIOS DE AUDITORÍA EXTERNA QUE SE REALIZAR...,SERVICIOS DE AUDITORÍA EXTERNA QUE SE REALIZAR...,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","299,034.0",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...
256435,API-VERACRUZ,DICTAMEN DE LOS ESTADOS FINANCIEROS.,DICTAMEN DE LOS ESTADOS FINANCIEROS.,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","34,488.5",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...
401267,ASIPONA-VERACRUZ,DICTAMINAR LOS ESTADOS FINANCIEROS DEL EJERCIC...,DICTAMINAR LOS ESTADOS FINANCIEROS DEL EJERCIC...,TGA961209CG6,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC","34,071.2",HTTPS://COMPRANET.HACIENDA.GOB.MX/ESOP/GUEST/G...


### Resultados de búsqueda por institución

In [28]:
pd.options.display.float_format = '{:,}'.format

ins_df=buscar.groupby(['siglas_de_la_institucion', 'institucion']).agg({'importe':'sum'}).reset_index()

ins_df.sort_values(by='importe', ascending=False, inplace=True)
#institucion.set_index('Institución', inplace=True)
ins_df

,siglas_de_la_institucion,institucion,importe
1,API-TUXPAN,"ADMINISTRACIÓN PORTUARIA INTEGRAL DE TUXPAN, S...","924,520.0"
0,API-SALINA CRUZ,ADMINISTRACIÓN PORTUARIA INTEGRAL DE SALINA CR...,"604,536.0"
2,API-VERACRUZ,"ADMINISTRACIÓN PORTUARIA INTEGRAL DE VERACRUZ,...","379,373.5"
3,ASIPONA-VERACRUZ,ADMINISTRACIÓN DEL SISTEMA PORTUARIO NACIONAL ...,"34,071.2"


### Resultados de búsqueda por proveedor

In [29]:
pd.options.display.float_format = '{:,}'.format

prov_df=buscar.groupby(['proveedor_o_contratista', 'rfc']).agg({'importe':'sum'}).reset_index()

prov_df.sort_values(by='importe', ascending=False, inplace=True)

def convert_year(two_digit_year):
    if int(two_digit_year) > 23:
        return "19" + two_digit_year
    else:
        return "20" + two_digit_year

prov_df['año_constitucion'] = prov_df['rfc'].astype(str).str[3:5].apply(lambda x: convert_year(x))

#institucion.set_index('Institución', inplace=True)
prov_df

,proveedor_o_contratista,rfc,importe,año_constitucion
0,"TRUEBA GRACIAN, ALTAMIRANO-DFK SC",TGA961209CG6,"1,942,500.7",1996
